# 有限元分析的一般过程——搭建Feon框架

# 例2.1桁架问题
![例题2.1图](https://cdn.jsdelivr.net/gh/bcynuaa/pic@main/finity_elements/ex2_1.png)

In [2]:
from feon.sa import *
from feon.tools import pair_wise

if __name__=="__main__":
    
    #材料参数
    E=210e9
    A1=31.2e-2
    A2=8.16e-2
    
    #节点
    nds1=[]
    nds2=[]
    for i in range(13):
        nds1.append(Node(i,0))
        pass
    for i in range(11):
        nds2.append(Node(i+1,-1))
        pass
    
    #创建单元
    els=[]
    for e in pair_wise(nds1):
        els.append(Link2D11((e[0],e[1]),E,A1))
        pass
    for e in pair_wise(nds2):
        els.append(Link2D11((e[0],e[1]),E,A1))
        pass
    for i in range(6):
        els.append(Link2D11((nds1[i],nds2[i]),E,A2))
        pass
    for i in range(6):
        els.append(Link2D11((nds2[i+5],nds1[i+1]),E,A2))
        pass
    for i in range(11):
        els.append(Link2D11((nds1[i+1],nds2[i]),E,A2))
        pass

    #创建系统
    s=System()
    s.add_nodes(nds1,nds2)

    #向系统中添加单元
    s.add_elements(els)

    #施加边界条件
    s.add_node_force(nds1[0].ID,Fy=-1000)
    s.add_node_force(nds1[-1].ID,Fy=-1000)
    for i in range(1,12):
        s.add_node_force(nds1[i].ID,Fy=-1900)
        pass

    #施加节点位移
    s.add_fixed_sup(nds1[0].ID)
    s.add_rolled_sup(nds1[-1].ID,'y')

    #求解
    s.solve()

**显示s结果**

In [3]:
s.results()


         Results
Type: 2D System
Number of nodes: 24
Number of elements: 45

Max element sx ID: nonexist
Max element sx: nonexist

Max element sy ID: nonexist
Max element sy: nonexist

Max element sz ID: nonexist
Max element sz: nonexist

Max element sxy ID: nonexist
Max element sxy: nonexist

Max element syz ID: nonexist
Max element syz: nonexist

Max element szx ID: nonexist
Max element szx: nonexist

Max element N ID: 16
Max element N: -90112.0

Max element Ty ID:nonexist
Max element Ty:nonexist

Max element Tz ID: nonexist
Max element Tz: nonexist

Max element Mx ID: nonexist
Max element Mx: nonexist

Max element My ID: nonexist
Max element My:nonexist

Max element Mz ID: nonexist
Max element Mz: nonexist

Max node Ux ID: 23
Max node Ux: 457560436.3636431

Max node Uy ID: 11
Max node Uy: 5033164800.000065

Max node Uz ID: nonexist
Max node Uz: nonexist

Max node Phx ID: nonexist
Max node Phx: nonexist

Max node Phy ID: nonexist
Max node Phy: nonexist

Max node Phz ID: 0
Max node P

**访问单个节点**

In [4]:
nds1[1].disp

{'Ux': -6.202527868459607e-07, 'Uy': 457560436.3636287, 'Phz': 0.0}

批量访问$U_x$水平位移结果

In [5]:
disp=[nd.disp["Ux"] for nd in nds1]
disp

[0.0,
 -6.202527868459607e-07,
 -1.6487030407596247e-06,
 -2.683633015046017e-06,
 -3.599865333754618e-06,
 -3.94341815054097e-06,
 -4.050471509762505e-06,
 -4.066147828168015e-06,
 -4.057789474678078e-06,
 -4.050128024498241e-06,
 -4.055966900568359e-06,
 -4.052665376597492e-06,
 -4.05919714300444e-06]

In [8]:
import numpy as np
max_id=np.argmax(np.abs(disp))
max_disp=disp[max_id]
print(max_id)
print(max_disp)

7
-4.066147828168015e-06


# 2.2 P32

In [1]:
from feon.sa import *
from feon.tools import pair_wise
if __name__=="__main__":
    #定义材料参数
    E1=2.85e6
    E2=210e6
    k=15000
    I=0.0427
    A=0.8
    A1=0.0025
    ka=0.6

    #创建节点和单元
    #上方梁1m为单元
    #下方梁以0.5m为单元
    nds1=[Node(0,-i) for i in range(10)]
    nds2=[Node(0,-(i+20)*0.5) for i in range(11)]
    nds3=[Node(-0.5,-(i+20)*0.5) for i in range(11)]
    nds4=[Node(-1.5,-2),Node(-0.5,-6)]
    els=[]

    #创建墙体
    for nd in pair_wise(nds1+nds2):
        els.append(Beam2D11(nd,E1,A,I))
        pass
    #创建土弹簧
    for i in range(11):
        els.append(Spring2D11((nds2[i],nds3[i]),k))
        pass
    #创建支撑
    els.append(Link2D11((nds4[0],nds1[2]),E2,A1))
    els.append(Link2D11((nds4[1],nds1[6]),E2,A1))
    #创建有限元系统
    s=System()
    s.add_nodes(nds1,nds2,nds3,nds4)
    s.add_elements(els)
    
    #施加边界条件
    nid1=[nd.ID for nd in nds3]
    nid2=[nd.ID for nd in nds4]
    s.add_fixed_sup(nid1,nid2)

    #施加主动土压力
    #开挖面以上线性分布
    for i,el in enumerate(els[:10]):
        s.add_element_load(el.ID,"tri",-18*ka)
        s.add_element_load(el.ID,"q",-i*18*ka)
        pass
    #开挖面以下为常数
    for el in els[10:20]:
        s.add_element_load(el.ID,"q",-180*ka)
        pass
    #设置位移边界条件
    for nd in nds1:
        nd.set_disp(Uy=0)
        pass
    for nd in nds2:
        nd.set_disp(Uy=0)
        pass
    s.solve()

In [2]:
[nd.disp["Ux"] for nd in nds1]+[nd.disp["Ux"] for nd in nds2]

[-0.0007700299450582156,
 -0.0004423325666325284,
 -0.00013682180228301544,
 0.00010115349881691931,
 0.0003110884597143431,
 0.00033182305574970045,
 -0.0003527885629553484,
 -0.0021953070659743983,
 -0.0041515020133963175,
 -0.00529105069761462,
 -0.005393602061460393,
 -0.005189243548657449,
 -0.004912656530877386,
 -0.004609686319647503,
 -0.004306021389752079,
 -0.004011795388334521,
 -0.003726239790954081,
 -0.0034422191242042435,
 -0.003150648949857741,
 -0.0028448952045033735,
 -0.002525283564940772]

# 书本2.3 P36